
---

# **Chapter 15: Redux & MobX**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Understand the Redux architecture (Store, Actions, Reducers, Middleware)
- Implement Redux in Flutter using the flutter_redux package
- Use Redux Thunk and Redux Saga for async operations
- Implement MobX with observables, actions, and reactions
- Use flutter_mobx widgets for reactive UI updates
- Generate MobX boilerplate using build_runner
- Compare Redux, MobX, BLoC, and Provider patterns
- Choose the appropriate state management solution for different scenarios

---

## **Prerequisites**

- Completed Chapter 14: BLoC Pattern
- Understanding of functional programming concepts
- Knowledge of Streams and reactive programming
- Familiarity with code generation tools
- Understanding of unidirectional data flow

---

## **15.1 Redux Architecture**

Redux is a predictable state container inspired by functional programming. It centralizes application state in a single immutable store, making state changes predictable and traceable.

### **Core Concepts**

```dart
// Redux follows three fundamental principles:
// 1. Single Source of Truth: One store holds entire app state
// 2. State is Read-Only: Only way to change state is emitting an action
// 3. Changes are Made with Pure Functions: Reducers specify how state changes

// STATE - Immutable application state
class AppState {
  final int counter;
  final List<String> items;
  final bool isLoading;
  final String? error;

  const AppState({
    this.counter = 0,
    this.items = const [],
    this.isLoading = false,
    this.error,
  });

  // Factory for initial state
  factory AppState.initial() => const AppState();

  // Copy method for immutability
  AppState copyWith({
    int? counter,
    List<String>? items,
    bool? isLoading,
    String? error,
  }) {
    return AppState(
      counter: counter ?? this.counter,
      items: items ?? this.items,
      isLoading: isLoading ?? this.isLoading,
      error: error, // Can set to null
    );
  }

  @override
  String toString() => 
      'AppState(counter: $counter, items: ${items.length}, loading: $isLoading)';
}

// ACTION - Plain objects describing what happened
// Actions are the only way to trigger state changes
abstract class AppAction {}

// Counter actions
class IncrementAction implements AppAction {
  final int amount;
  IncrementAction({this.amount = 1});
}

class DecrementAction implements AppAction {
  final int amount;
  DecrementAction({this.amount = 1});
}

// Item actions
class AddItemAction implements AppAction {
  final String item;
  AddItemAction(this.item);
}

class RemoveItemAction implements AppAction {
  final String item;
  RemoveItemAction(this.item);
}

// Async actions (using Thunk pattern)
class LoadItemsAction implements AppAction {
  final List<String> items;
  LoadItemsAction(this.items);
}

class SetLoadingAction implements AppAction {
  final bool isLoading;
  SetLoadingAction(this.isLoading);
}

class SetErrorAction implements AppAction {
  final String? error;
  SetErrorAction(this.error);
}

// REDUCER - Pure function: (State, Action) => State
// Reducers specify how state changes in response to actions
AppState appReducer(AppState state, dynamic action) {
  // Redux reduces use switch or if-else to handle different actions
  // Must return new state, never mutate existing state
  
  if (action is IncrementAction) {
    // Return new state with incremented counter
    return state.copyWith(counter: state.counter + action.amount);
  } 
  else if (action is DecrementAction) {
    return state.copyWith(counter: state.counter - action.amount);
  } 
  else if (action is AddItemAction) {
    // Create new list with added item (immutable update)
    return state.copyWith(items: [...state.items, action.item]);
  } 
  else if (action is RemoveItemAction) {
    return state.copyWith(
      items: state.items.where((item) => item != action.item).toList(),
    );
  } 
  else if (action is LoadItemsAction) {
    return state.copyWith(
      items: action.items,
      isLoading: false,
      error: null,
    );
  } 
  else if (action is SetLoadingAction) {
    return state.copyWith(isLoading: action.isLoading);
  } 
  else if (action is SetErrorAction) {
    return state.copyWith(error: action.error, isLoading: false);
  }
  
  // Return unchanged state for unknown actions
  return state;
}

// Combine multiple reducers (for larger apps)
AppState combinedReducer(AppState state, dynamic action) {
  return AppState(
    counter: counterReducer(state.counter, action),
    items: itemsReducer(state.items, action),
    isLoading: loadingReducer(state.isLoading, action),
    error: errorReducer(state.error, action),
  );
}

int counterReducer(int state, dynamic action) {
  if (action is IncrementAction) return state + action.amount;
  if (action is DecrementAction) return state - action.amount;
  return state;
}

List<String> itemsReducer(List<String> state, dynamic action) {
  if (action is AddItemAction) return [...state, action.item];
  if (action is RemoveItemAction) {
    return state.where((item) => item != action.item).toList();
  }
  if (action is LoadItemsAction) return action.items;
  return state;
}
```

**Explanation:**

- **Store**: The single source of truth that holds the entire application state. In Redux, there's only one store.
- **State**: Immutable data structure representing the UI at a specific point in time. Never modify state directly; always create new instances.
- **Actions**: Plain objects (classes) that describe *what happened* (e.g., user clicked button, data loaded). They must have a `type` (implemented via class types in Dart).
- **Reducers**: Pure functions that take the current state and an action, and return a new state. They must be pure (no side effects, same input always produces same output).
- **Immutability**: The `...` spread operator creates new lists/objects rather than modifying existing ones. This enables efficient change detection and time-travel debugging.
- **Combined Reducers**: For large apps, split reducers by domain (user, cart, settings) and combine them.

### **Redux Store and Middleware**

```dart
import 'package:redux/redux.dart';
import 'package:redux_thunk/redux_thunk.dart';

// MIDDLEWARE - Extension point between dispatching an action and the moment it reaches the reducer
// Used for logging, crash reporting, async operations, routing

// Logging Middleware
void loggingMiddleware(
  Store<AppState> store,
  dynamic action,
  NextDispatcher next,
) {
  // next(action) passes action to next middleware or reducer
  print('${DateTime.now()}: Action: $action');
  print('Previous State: ${store.state}');
  
  next(action); // Pass to next middleware/reducer
  
  print('New State: ${store.state}');
}

// Thunk Middleware for async actions
// Thunk is a function that takes (store) and performs async work
ThunkAction<AppState> fetchItems() {
  return (Store<AppState> store) async {
    // Dispatch loading action
    store.dispatch(SetLoadingAction(true));
    
    try {
      // Simulate API call
      await Future.delayed(Duration(seconds: 1));
      final items = ['Item 1', 'Item 2', 'Item 3'];
      
      // Dispatch success action with data
      store.dispatch(LoadItemsAction(items));
    } catch (e) {
      // Dispatch error action
      store.dispatch(SetErrorAction(e.toString()));
    }
  };
}

// Creating the Store
final store = Store<AppState>(
  appReducer,
  initialState: AppState.initial(),
  middleware: [
    loggingMiddleware, // Custom middleware
    thunkMiddleware,   // For async actions
  ],
  // Optional: Add dev tools in debug mode
  // distinct: true, // Only notify if state actually changed
);

void main() {
  // Dispatch actions
  store.dispatch(IncrementAction(amount: 5));
  store.dispatch(AddItemAction('New Item'));
  
  // Dispatch async action (thunk)
  store.dispatch(fetchItems());
  
  // Listen to state changes
  store.onChange.listen((state) {
    print('State updated: $state');
  });
}
```

**Explanation:**

- **Store Creation**: The `Store` takes the root reducer, initial state, and middleware array.
- **Middleware**: Functions that intercept every action. They can modify, delay, log, or transform actions before they reach reducers.
- **`next(action)`**: Middleware must call `next(action)` to pass the action down the chain. Forgetting this stops the action.
- **Thunk Middleware**: Allows dispatching functions (thunks) instead of plain objects. Thunks contain async logic and can dispatch multiple actions.
- **Async Flow**: Thunk dispatches loading → API call → dispatches success/error → reducer updates state.
- **Subscriptions**: `store.onChange` is a Stream that emits new states whenever the store updates.

---

## **15.2 Flutter Redux Integration**

The flutter_redux package connects Redux stores to Flutter widgets.

```dart
import 'package:flutter/material.dart';
import 'package:flutter_redux/flutter_redux.dart';
import 'package:redux/redux.dart';

void main() {
  final store = Store<AppState>(
    appReducer,
    initialState: AppState.initial(),
  );
  
  runApp(MyApp(store: store));
}

class MyApp extends StatelessWidget {
  final Store<AppState> store;
  
  const MyApp({Key? key, required this.store}) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    return StoreProvider<AppState>(
      // Makes Redux store available to all descendant widgets
      store: store,
      child: MaterialApp(
        title: 'Redux Demo',
        home: HomeScreen(),
      ),
    );
  }
}

class HomeScreen extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Redux Counter')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            // StoreConnector connects to store and rebuilds on changes
            StoreConnector<AppState, int>(
              // converter: Extracts data from store state
              converter: (store) => store.state.counter,
              // builder: Builds widget with converted data
              builder: (context, count) {
                return Text(
                  'Count: $count',
                  style: Theme.of(context).textTheme.headline2,
                );
              },
            ),
            
            SizedBox(height: 20),
            
            // Accessing list data
            StoreConnector<AppState, List<String>>(
              converter: (store) => store.state.items,
              builder: (context, items) {
                return Text('Items: ${items.length}');
              },
            ),
            
            SizedBox(height: 20),
            
            // Combined data with view model
            StoreConnector<AppState, _ViewModel>(
              converter: (store) => _ViewModel(
                counter: store.state.counter,
                isLoading: store.state.isLoading,
                onIncrement: () => store.dispatch(IncrementAction()),
                onDecrement: () => store.dispatch(DecrementAction()),
                onAddItem: (item) => store.dispatch(AddItemAction(item)),
              ),
              builder: (context, viewModel) {
                if (viewModel.isLoading) {
                  return CircularProgressIndicator();
                }
                return Column(
                  children: [
                    Text('Counter: ${viewModel.counter}'),
                    ElevatedButton(
                      onPressed: viewModel.onIncrement,
                      child: Text('Increment'),
                    ),
                  ],
                );
              },
            ),
          ],
        ),
      ),
      floatingActionButton: FloatingActionButton(
        onPressed: () {
          // Access store from anywhere using StoreProvider
          StoreProvider.of<AppState>(context).dispatch(
            AddItemAction('Item ${DateTime.now()}'),
          );
        },
        child: Icon(Icons.add),
      ),
    );
  }
}

// ViewModel class to bundle data and callbacks
class _ViewModel {
  final int counter;
  final bool isLoading;
  final VoidCallback onIncrement;
  final VoidCallback onDecrement;
  final Function(String) onAddItem;
  
  _ViewModel({
    required this.counter,
    required this.isLoading,
    required this.onIncrement,
    required this.onDecrement,
    required this.onAddItem,
  });
}

// Optimization: Prevent unnecessary rebuilds
class OptimizedCounter extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return StoreConnector<AppState, int>(
      converter: (store) => store.state.counter,
      // distinct: Only rebuild if converter result changes
      distinct: true,
      // onInit: Called when widget is first inserted
      onInit: (store) => print('Counter widget initialized'),
      // onDispose: Called when widget is removed
      onDispose: (store) => print('Counter widget disposed'),
      // onWillChange: Called before builder with old and new view models
      onWillChange: (prevViewModel, newViewModel) {
        print('Counter changing from $prevViewModel to $newViewModel');
      },
      // onDidChange: Called after widget rebuilds
      onDidChange: (viewModel, context) {
        // Good for side effects like showing snackbars
        ScaffoldMessenger.of(context).showSnackBar(
          SnackBar(content: Text('Count updated to $viewModel')),
        );
      },
      builder: (context, count) {
        print('Building counter: $count');
        return Text('$count');
      },
    );
  }
}
```

**Explanation:**

- **`StoreProvider`**: InheritedWidget that makes the Redux store available to the widget tree. Must wrap the app or relevant subtree.
- **`StoreConnector`**: Primary widget for connecting to Redux. It:
  - Subscribes to store updates
  - Runs `converter` to extract relevant data
  - Rebuilds `builder` when extracted data changes
- **`converter`**: Function that transforms the full AppState into the specific data the widget needs. This is like `mapStateToProps` in React-Redux.
- **`distinct`**: If true, only rebuilds if the converter result changes (uses `==` comparison).
- **ViewModel Pattern**: Bundle data and action dispatchers into a single object for cleaner builder methods.
- **Lifecycle hooks**: `onInit`, `onDispose`, `onWillChange`, `onDidChange` allow side effects at specific points.
- **Accessing store**: `StoreProvider.of<AppState>(context)` gets the store directly for dispatching actions outside of StoreConnector.

### **Redux Architecture with Multiple Features**

```dart
// For larger apps, organize by feature/domain

// FEATURE: Authentication
// auth_actions.dart
class LoginRequest implements AppAction {
  final String email;
  final String password;
  LoginRequest(this.email, this.password);
}

class LoginSuccess implements AppAction {
  final String token;
  final User user;
  LoginSuccess(this.token, this.user);
}

class LoginFailure implements AppAction {
  final String error;
  LoginFailure(this.error);
}

class Logout implements AppAction {}

// auth_state.dart
class AuthState {
  final bool isAuthenticated;
  final User? user;
  final String? token;
  final bool isLoading;
  final String? error;

  AuthState({
    this.isAuthenticated = false,
    this.user,
    this.token,
    this.isLoading = false,
    this.error,
  });

  AuthState copyWith({...}) => AuthState(...);
}

// auth_reducer.dart
AuthState authReducer(AuthState state, dynamic action) {
  if (action is LoginRequest) {
    return state.copyWith(isLoading: true, error: null);
  }
  if (action is LoginSuccess) {
    return state.copyWith(
      isAuthenticated: true,
      user: action.user,
      token: action.token,
      isLoading: false,
    );
  }
  if (action is LoginFailure) {
    return state.copyWith(isLoading: false, error: action.error);
  }
  if (action is Logout) {
    return AuthState(); // Reset to initial
  }
  return state;
}

// Root state combining all features
class AppState {
  final AuthState auth;
  final CartState cart;
  final SettingsState settings;
  
  AppState({
    required this.auth,
    required this.cart,
    required this.settings,
  });
  
  factory AppState.initial() => AppState(
    auth: AuthState(),
    cart: CartState(),
    settings: SettingsState(),
  );
}

// Root reducer combining all reducers
AppState appReducer(AppState state, dynamic action) {
  return AppState(
    auth: authReducer(state.auth, action),
    cart: cartReducer(state.cart, action),
    settings: settingsReducer(state.settings, action),
  );
}
```

**Explanation:**

- **Feature-based organization**: Split actions, state, and reducers by domain (auth, cart, settings).
- **Combined State**: Root AppState contains substates for each feature.
- **Combined Reducer**: Each feature reducer handles its own substate. Action types can be handled by multiple reducers if needed.
- **Decoupling**: Features are independent; they only interact through the global store.

---

## **15.3 MobX Reactive State Management**

MobX uses observables and reactions for automatic UI updates. It's inspired by reactive programming and Transparent Functional Reactive Programming (TFRP).

### **Core Concepts**

```dart
import 'package:mobx/mobx.dart';

// MobX uses three core concepts:
// 1. OBSERVABLES: State that can be observed
// 2. ACTIONS: Methods that modify observables
// 3. REACTIONS: Side effects that run when observables change

// Define a Store class
class Counter {
  // OBSERVABLE: Mark fields that should trigger reactions when changed
  @observable
  int value = 0;
  
  @observable
  String name = 'Counter';
  
  // COMPUTED: Derived values that update automatically when dependencies change
  @computed
  String get displayValue => '$name: $value';
  
  // COMPUTED with logic
  @computed
  bool get isEven => value % 2 == 0;
  
  // ACTION: Methods that modify observables
  // Actions batch updates and notify reactions only once
  @action
  void increment() {
    value++;
  }
  
  @action
  void decrement() {
    value--;
  }
  
  @action
  void setName(String newName) {
    name = newName;
  }
  
  // Async action
  @action
  Future<void> incrementAsync() async {
    await Future.delayed(Duration(seconds: 1));
    value++;
  }
  
  // Action modifying multiple observables (batched into single notification)
  @action
  void reset() {
    value = 0;
    name = 'Reset';
    // Both changes trigger only one reaction cycle
  }
}
```

**Explanation:**

- **`@observable`**: Marks a field as observable. When the value changes, all reactions (observers) are notified.
- **`@computed`**: Read-only properties that automatically cache and update when their dependencies change. Like derived state in other patterns.
- **`@action`**: Methods that modify observables. Actions batch multiple observable changes into a single notification, preventing excessive rebuilds.
- **Reactivity**: Unlike Redux where you manually dispatch actions, MobX automatically tracks dependencies and updates observers.
- **No boilerplate**: Compared to Redux (Actions + Reducers), MobX just needs observables and actions.

### **Code Generation Setup**

```dart
// MobX uses code generation for Dart language limitations
// Setup:

// 1. Add to pubspec.yaml:
// dependencies:
//   mobx: ^2.0.0
//   flutter_mobx: ^2.0.0
//
// dev_dependencies:
//   build_runner: ^2.0.0
//   mobx_codegen: ^2.0.0

// 2. Create store file with .g.dart part directive
import 'package:mobx/mobx.dart';

part 'counter.g.dart'; // Generated file

// 3. Make class abstract with Store mixin
class Counter = _Counter with _$Counter;

abstract class _Counter with Store {
  @observable
  int value = 0;
  
  @action
  void increment() => value++;
}

// 4. Run code generation:
// dart run build_runner build     (one-time)
// dart run build_runner watch     (continuous during development)

// GENERATED FILE (counter.g.dart) - Auto-generated, don't edit
/*
part of 'counter.dart';

mixin _$Counter on _Counter, Store {
  late final _$valueAtom = Atom(name: '_Counter.value', context: context);

  @override
  int get value {
    _$valueAtom.reportRead();
    return super.value;
  }

  @override
  set value(int value) {
    _$valueAtom.reportWrite(value, super.value, () {
      super.value = value;
    });
  }

  late final _$_CounterActionController = ActionController(name: '_Counter');

  @override
  void increment() {
    final _$actionInfo = _$_CounterActionController.startAction(name: '_Counter.increment');
    try {
      return super.increment();
    } finally {
      _$_CounterActionController.endAction(_$actionInfo);
    }
  }
}
*/
```

**Explanation:**

- **Code Generation**: Dart doesn't support annotations modifying behavior at runtime like JavaScript/TypeScript. MobX uses `build_runner` to generate boilerplate.
- **Abstract Class Pattern**: Define abstract class with `_` prefix, then create concrete class mixing in the generated class.
- **Atom**: Generated code creates `Atom` objects for each observable. These track reads (reportRead) and writes (reportWrite).
- **ActionController**: Wraps action methods to batch notifications.
- **Generated file**: `counter.g.dart` contains the reactive implementation. Never edit this file manually.

---

## **15.4 Flutter MobX Integration**

The flutter_mobx package provides widgets that rebuild when observables change.

```dart
import 'package:flutter/material.dart';
import 'package:flutter_mobx/flutter_mobx.dart';

class CounterScreen extends StatelessWidget {
  final Counter counter = Counter(); // Instantiate store
  
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('MobX Counter')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            // OBSERVER: Widget that rebuilds when observables inside change
            Observer(
              builder: (_) {
                // This builder runs every time counter.value changes
                print('Building counter value: ${counter.value}');
                return Text(
                  '${counter.value}',
                  style: Theme.of(context).textTheme.headline2,
                );
              },
            ),
            
            SizedBox(height: 20),
            
            // Observer for computed value
            Observer(
              builder: (_) => Text(
                counter.displayValue,
                style: TextStyle(fontSize: 24),
              ),
            ),
            
            SizedBox(height: 10),
            
            // Observer for another computed
            Observer(
              builder: (_) => Chip(
                label: Text(counter.isEven ? 'Even' : 'Odd'),
                backgroundColor: counter.isEven ? Colors.green : Colors.orange,
              ),
            ),
            
            SizedBox(height: 20),
            
            // TextField bound to observable
            TextField(
              onChanged: (value) => counter.setName(value),
              decoration: InputDecoration(labelText: 'Counter Name'),
            ),
          ],
        ),
      ),
      floatingActionButton: Column(
        mainAxisAlignment: MainAxisAlignment.end,
        children: [
          FloatingActionButton(
            onPressed: counter.increment, // Direct method call
            child: Icon(Icons.add),
          ),
          SizedBox(height: 10),
          FloatingActionButton(
            onPressed: counter.decrement,
            child: Icon(Icons.remove),
          ),
          SizedBox(height: 10),
          FloatingActionButton(
            onPressed: counter.incrementAsync, // Async action
            child: Icon(Icons.timer),
          ),
        ],
      ),
    );
  }
}
```

**Explanation:**

- **`Observer`**: Widget that tracks which observables are accessed in its builder. Rebuilds automatically when any of those observables change.
- **Granular updates**: Unlike Redux where the whole StoreConnector rebuilds, Observer only rebuilds if its specific observables change.
- **Automatic tracking**: You don't specify which observables to watch; MobX automatically tracks access during the build method.
- **Method calls**: Call action methods directly (`counter.increment()`). No need to dispatch actions or go through reducers.
- **Async support**: Actions can be async, and observables updated after awaits trigger rebuilds.

### **Complex MobX Store Example**

```dart
import 'package:mobx/mobx.dart';

part 'todo_store.g.dart';

class TodoStore = _TodoStore with _$TodoStore;

abstract class _TodoStore with Store {
  // Observable list - special handling for collections
  @observable
  ObservableList<Todo> todos = ObservableList<Todo>();
  
  @observable
  String filter = 'all'; // 'all', 'completed', 'active'
  
  @observable
  bool isLoading = false;
  
  // Computed filtered list
  @computed
  List<Todo> get filteredTodos {
    switch (filter) {
      case 'completed':
        return todos.where((todo) => todo.completed).toList();
      case 'active':
        return todos.where((todo) => !todo.completed).toList();
      case 'all':
      default:
        return todos.toList();
    }
  }
  
  @computed
  int get completedCount => todos.where((t) => t.completed).length;
  
  @computed
  int get pendingCount => todos.length - completedCount;
  
  @computed
  bool get hasCompletedTodos => completedCount > 0;
  
  // Actions
  @action
  void setFilter(String newFilter) {
    filter = newFilter;
  }
  
  @action
  void addTodo(String title) {
    final todo = Todo(
      id: DateTime.now().toString(),
      title: title,
    );
    todos.add(todo);
  }
  
  @action
  void toggleTodo(String id) {
    final todo = todos.firstWhere((t) => t.id == id);
    todo.completed = !todo.completed; // Todo class must also use MobX
  }
  
  @action
  void removeTodo(String id) {
    todos.removeWhere((t) => t.id == id);
  }
  
  @action
  Future<void> loadTodos() async {
    isLoading = true;
    
    try {
      await Future.delayed(Duration(seconds: 1));
      // Mock data
      todos.addAll([
        Todo(id: '1', title: 'Learn MobX', completed: true),
        Todo(id: '2', title: 'Build app', completed: false),
      ]);
    } finally {
      isLoading = false;
    }
  }
  
  @action
  void clearCompleted() {
    todos.removeWhere((t) => t.completed);
  }
}

// Nested observable class
class Todo = _Todo with _$Todo;

abstract class _Todo with Store {
  final String id;
  
  @observable
  String title;
  
  @observable
  bool completed;
  
  _Todo({
    required this.id,
    required this.title,
    this.completed = false,
  });
}
```

**Explanation:**

- **`ObservableList`**: Regular List isn't deeply observable. Use `ObservableList` to detect additions/removals/reordering.
- **Deep observability**: If Todo class also uses MobX (@observable fields), changes to individual todo items trigger reactions.
- **Multiple computeds**: Define multiple derived values (completedCount, pendingCount). They cache results and only recalculate when dependencies change.
- **Actions with logic**: Complex logic (filtering, finding items) lives in the store, not the UI.

---

## **15.5 MobX Reactions (Side Effects)**

Reactions run side effects when observables change, similar to BlocListener in BLoC.

```dart
import 'package:mobx/mobx.dart';

void setupReactions(TodoStore store) {
  // 1. AUTORUN: Runs immediately and whenever any observable inside changes
  final disposeAutorun = autorun((_) {
    print('Todos count: ${store.todos.length}');
    print('Completed: ${store.completedCount}');
    // Runs on setup and every time todos or completedCount changes
  });
  
  // 2. REACTION: Tracks specific observable, runs effect when predicate changes
  final disposeReaction = reaction(
    // Predicate: What to track
    (_) => store.hasCompletedTodos,
    // Effect: What to do when predicate result changes
    (hasCompleted) {
      if (hasCompleted) {
        print('First completed todo created!');
        // Could show notification here
      }
    },
    // Optional: Fire immediately with initial value
    fireImmediately: true,
  );
  
  // 3. WHEN: Runs once when condition becomes true, then disposes
  final disposeWhen = when(
    // Condition
    (_) => store.pendingCount == 0 && store.todos.isNotEmpty,
    // Action
    () {
      print('All todos completed! Great job!');
      // Show celebration, enable "clear all" button, etc.
    },
  );
  
  // 4. INTERCEPT: Modify or prevent state changes (middleware-like)
  final disposeIntercept = store.todos.intercept((change) {
    // Intercept list modifications
    print('About to modify todos: ${change.type}');
    
    // Can modify the change
    if (change.type == OperationType.add) {
      final newTodo = change.newValue as Todo;
      if (newTodo.title.isEmpty) {
        print('Preventing empty todo');
        return null; // Cancel the change
      }
    }
    
    return change; // Proceed with change
  });
  
  // Cleanup (call in dispose)
  // disposeAutorun();
  // disposeReaction();
  // disposeWhen(); // Not needed for when, auto-disposes
}

// Flutter integration with reactions
class TodoScreen extends StatefulWidget {
  @override
  _TodoScreenState createState() => _TodoScreenState();
}

class _TodoScreenState extends State<TodoScreen> {
  late final TodoStore store;
  List<ReactionDisposer>? _disposers;
  
  @override
  void initState() {
    super.initState();
    store = TodoStore();
    
    // Setup reactions in initState
    _disposers = [
      reaction(
        (_) => store.hasCompletedTodos,
        (hasCompleted) {
          if (hasCompleted) {
            ScaffoldMessenger.of(context).showSnackBar(
              SnackBar(content: Text('You have completed todos!')),
            );
          }
        },
      ),
    ];
  }
  
  @override
  void dispose() {
    // Clean up reactions to prevent memory leaks
    _disposers?.forEach((d) => d());
    super.dispose();
  }
  
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      body: Observer(
        builder: (_) => ListView.builder(
          itemCount: store.filteredTodos.length,
          itemBuilder: (context, index) {
            final todo = store.filteredTodos[index];
            return ListTile(
              title: Text(todo.title),
              trailing: Checkbox(
                value: todo.completed,
                onChanged: (_) => store.toggleTodo(todo.id),
              ),
            );
          },
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`autorun`**: Immediately runs the passed function and re-runs whenever any observable accessed inside changes. Good for logging.
- **`reaction`**: Tracks a specific value (predicate) and runs effect only when that specific value changes. More efficient than autorun.
- **`when`**: Runs effect once when predicate returns true, then automatically disposes. Good for one-time actions.
- **`intercept`**: Intercepts changes to observables, allowing validation, transformation, or cancellation (like middleware).
- **Disposers**: Reactions return functions that must be called to clean up and prevent memory leaks.
- **Flutter lifecycle**: Setup reactions in `initState`, dispose in `dispose`. This prevents widget memory leaks.

---

## **15.6 Choosing the Right State Management**

Comparison and decision guide for all patterns covered.

```dart
// DECISION MATRIX

// PROVIDER (Chapter 12)
// Best for: Beginners, simple apps, migrating from setState
// Pros: Simple, officially recommended, good documentation
// Cons: Can become verbose with complex state, lacks compile-time safety
// Use when: Learning Flutter, small to medium apps, team is new to Flutter

// RIVERPOD (Chapter 13)
// Best for: All app sizes, especially medium to large
// Pros: Compile-safe, auto-dispose, testable, no BuildContext needed
// Cons: Newer pattern, learning curve for Provider users
// Use when: Starting new projects, want Provider benefits without drawbacks

// BLoC (Chapter 14)
// Best for: Large apps, complex business logic, teams familiar with reactive patterns
// Pros: Excellent separation of concerns, testable, predictable, event history
// Cons: Boilerplate-heavy, steep learning curve, overkill for simple apps
// Use when: Complex apps with many features, need event sourcing, large teams

// REDUX (Chapter 15)
// Best for: Teams coming from React/Redux, need time-travel debugging
// Pros: Predictable state flow, excellent DevTools, single source of truth
// Cons: Lots of boilerplate (actions, reducers, middleware), verbose
// Use when: Team has Redux experience, need advanced debugging, strict architecture

// MobX (Chapter 15)
// Best for: Complex reactive UIs, teams familiar with OOP reactive patterns
// Pros: Minimal boilerplate, automatic tracking, fine-grained updates
// Cons: Code generation required, "magic" can be confusing, harder to debug
// Use when: Complex forms with many interdependent fields, need OOP approach

// PRACTICAL EXAMPLES:

// Scenario 1: Simple Counter App
// Choice: Riverpod or Provider
// Why: Simple state, no complex logic needed

// Scenario 2: E-commerce App with Cart, Auth, Catalog
// Choice: Riverpod or BLoC
// Why: Multiple features, need clear separation, async operations

// Scenario 3: Banking/Finance App with strict audit requirements
// Choice: Redux or BLoC with HydratedBloc
// Why: Need event history, time-travel debugging, state persistence

// Scenario 4: Social Media Feed with Real-time Updates
// Choice: Riverpod (StreamProvider) or MobX
// Why: Stream handling, reactive updates, complex UI dependencies

// Scenario 5: Enterprise App with 20+ Developers
// Choice: BLoC or Redux
// Why: Strict architecture enforcement, testability, scalability

// HYBRID APPROACH (Common in production):
// - Use Riverpod for global state (auth, theme, settings)
// - Use BLoC/MobX for complex features (checkout flow, multi-step forms)
// - Use Provider for simple dependency injection
// - Use setState for truly local UI state (animations, checkbox toggles)
```

**Explanation:**

- **Provider/Riverpod**: Best general-purpose choices. Riverpod is essentially Provider 2.0 with fixes for common issues.
- **BLoC**: Best when you need strict architectural boundaries and event sourcing. Common in large enterprise apps.
- **Redux**: Best for teams with web/React background. The DevTools are excellent for debugging.
- **MobX**: Best for complex reactive UIs where you want minimal boilerplate and automatic updates.
- **Hybrid**: Most production apps use multiple patterns. Don't force one pattern everywhere; use the right tool for each job.

---

## **Chapter Summary**

In this chapter, we covered Redux and MobX state management:

### **Key Takeaways:**

1. **Redux**:
   - Single immutable store containing all app state
   - Actions describe events, Reducers specify state changes
   - Middleware handles async operations (Thunks)
   - `StoreProvider` and `StoreConnector` for Flutter integration
   - Excellent for predictable state flow and debugging

2. **MobX**:
   - Observable state with automatic dependency tracking
   - Actions modify state, Reactions handle side effects
   - `@observable`, `@computed`, `@action` annotations
   - Code generation required (`build_runner`)
   - `Observer` widget for reactive UI updates
   - Fine-grained updates without manual subscription

3. **Comparison**:
   - **Boilerplate**: Redux (high) > BLoC (medium) > MobX (low) > Riverpod (low)
   - **Learning Curve**: Redux/BLoC (steep) > MobX (medium) > Riverpod (gentle)
   - **Debugging**: Redux (excellent DevTools) > BLoC (good) > MobX (moderate) > Riverpod (basic)
   - **Performance**: All are performant; MobX has finest-grained updates

4. **When to Choose**:
   - **Redux**: Large teams, need strict architecture, event sourcing requirements
   - **MobX**: Complex reactive forms, OOP preference, minimal boilerplate desire
   - **Riverpod**: Default choice for new Flutter projects
   - **BLoC**: Complex business logic, need event history

---

**End of Chapter 15**

---

# **Next Chapter: Chapter 16 - Navigation 2.0**

Chapter 16 will explore Flutter's declarative navigation system (Navigator 2.0), including Router, RouteInformationParser, and deep linking. You'll learn how to implement URL-based navigation for web support and complex navigation flows with authentication guards and nested routing.